In [2]:
import tensorflow as tf

In [3]:
a = tf.Variable(1.0)
b = tf.Variable(2.0)

@tf.function
def f(x,y):
    a.assign(y * b)
    b.assign_add(x * a)
    return a + b

print(f(1.0, 2.0))

print(tf.autograph.to_code(f.python_function))


tf.Tensor(10.0, shape=(), dtype=float32)
def tf__f(x, y):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('f', 'f_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as f_scope:
    ag__.converted_call(a.assign, f_scope.callopts, (y * b,), None, f_scope)
    ag__.converted_call(b.assign_add, f_scope.callopts, (x * a,), None, f_scope)
    do_return = True
    retval_ = f_scope.mark_return_value(a + b)
  do_return,
  return ag__.retval(retval_)



In [4]:
@tf.function
def sign(x):
    if x > 0:
        return 'Positive'
    else:
        return 'Negative or zero'

print("Sign = {}".format(sign(tf.constant(2))))
print("Sign = {}".format(sign(tf.constant(-2))))

print(tf.autograph.to_code(sign.python_function))


Sign = b'Positive'
Sign = b'Negative or zero'
def tf__sign(x):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('sign', 'sign_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as sign_scope:

    def get_state():
      return ()

    def set_state(_):
      pass

    def if_true():
      do_return = True
      retval_ = sign_scope.mark_return_value('Positive')
      return do_return, retval_

    def if_false():
      do_return = True
      retval_ = sign_scope.mark_return_value('Negative or zero')
      return do_return, retval_
    cond = x > 0
    do_return, retval_ = ag__.if_stmt(cond, if_true, if_false, get_state, set_state, ('do_return', 'retval_'), ())
  do_return,
  return ag__.retval(retval_)



In [5]:
@tf.function
def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        x = tf.tanh(x)
    return x

print(tf.autograph.to_code(f.python_function))


def tf__f(x):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('f', 'f_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as f_scope:

    def get_state():
      return ()

    def set_state(_):
      pass

    def loop_body(x):
      ag__.converted_call(tf.print, f_scope.callopts, (x,), None, f_scope)
      x = ag__.converted_call(tf.tanh, f_scope.callopts, (x,), None, f_scope)
      return x,

    def loop_test(x):
      return ag__.converted_call(tf.reduce_sum, f_scope.callopts, (x,), None, f_scope) > 1
    x, = ag__.while_stmt(loop_test, loop_body, get_state, set_state, (x,), ('x',), ())
    do_return = True
    retval_ = f_scope.mark_return_value(x)
  do_return,
  return ag__.retval(retval_)



In [6]:
@tf.function
def sum_even(items):
    s = 0
    for c in items:
        if c % 2 > 0:
            continue
        s += c
    return s

print(tf.autograph.to_code(sum_even.python_function))


def tf__sum_even(items):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('sum_even', 'sum_even_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as sum_even_scope:
    s = 0

    def get_state_2():
      return ()

    def set_state_2(_):
      pass

    def loop_body(iterates, s):
      c = iterates
      continue_ = False

      def get_state():
        return ()

      def set_state(_):
        pass

      def if_true():
        continue_ = True
        return continue_

      def if_false():
        return continue_
      cond = c % 2 > 0
      continue_ = ag__.if_stmt(cond, if_true, if_false, get_state, set_state, ('continue_',), ())

      def get_state_1():
        return ()

      def set_state_1(_):
        pass

      def if_true_1():
        s_1, = s,
        s_1 += c
        return s_1

      def if_false_1():
        return s
      cond_1 = ag__.not_(continue_)
 

## Print statements

Tracing also behaves differently in graph mode. First, here is a function (not decorated with @tf.function yet) that prints the value of the input parameter. f(2) is called in a for loop 5 times, and then f(3) is called.

In [8]:
def f(x):
    print("Traced with", x)

for i in range(5):
    f(2)
    
f(3)


Traced with 2
Traced with 2
Traced with 2
Traced with 2
Traced with 2
Traced with 3


If you were to decorate this function with @tf.function and run it, notice that the print statement only appears once for f(2) even though it is called in a loop.

In [9]:
@tf.function
def f(x):
    print("Traced with", x)

for i in range(5):
    f(2)
    
f(3)


Traced with 2
Traced with 3


## Now compare print to tf.print.

tf.print is graph aware and will run as expected in loops.
Try running the same code where tf.print() is added in addition to the regular print.

Note how tf.print behaves compared to print in graph mode.


In [10]:
@tf.function
def f(x):
    print("Traced with", x)
    # added tf.print
    tf.print("Executed with", x)

for i in range(5):
    f(2)
    
f(3)

Traced with 2
Executed with 2
Executed with 2
Executed with 2
Executed with 2
Executed with 2
Traced with 3
Executed with 3


## Avoid defining variables inside the function

This function (not decorated yet) defines a tensor v and adds the input x to it.

Here, it runs fine.



In [12]:
def f(x):
    v = tf.Variable(1.0)
    v.assign_add(x)
    return v

print(f(1))


<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>


Now if you decorate the function with @tf.function.

The cell below will throw an error because tf.Variable is defined within the function. The graph mode function should only contain operations.



In [15]:
@tf.function
def f(x):
    v = tf.Variable(1.0)
    v.assign_add(x)
    return v

print(f(1))

ValueError: in converted code:

    <ipython-input-13-5729586b3383>:3 f  *
        v = tf.Variable(1.0)
    C:\Users\Admin\anaconda3\envs\tfcpu\lib\site-packages\tensorflow_core\python\ops\variables.py:260 __call__
        return cls._variable_v2_call(*args, **kwargs)
    C:\Users\Admin\anaconda3\envs\tfcpu\lib\site-packages\tensorflow_core\python\ops\variables.py:254 _variable_v2_call
        shape=shape)
    C:\Users\Admin\anaconda3\envs\tfcpu\lib\site-packages\tensorflow_core\python\ops\variables.py:65 getter
        return captured_getter(captured_previous, **kwargs)
    C:\Users\Admin\anaconda3\envs\tfcpu\lib\site-packages\tensorflow_core\python\eager\def_function.py:413 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.


To get around the error above, simply move v = tf.Variable(1.0) to the top of the cell before the @tf.function decorator.

In [14]:
# define the variables outside of the decorated function
v = tf.Variable(1.0)

@tf.function
def f(x):
    return v.assign_add(x)

print(f(5))

tf.Tensor(6.0, shape=(), dtype=float32)
